In [ ]:
from pathlib import Path
import pandas as pd
from pandarallel import pandarallel
import wfdb
from wfdb.io import Record
from typing import List
import shutil

In [ ]:
pandarallel.initialize(progress_bar=True)

# Create folder for dataset
dataset_path = Path("../ctg/dataset")
dataset_path.mkdir(exist_ok=True)

In [ ]:
# Read the metadata file
dataset_records = pd.read_csv("../ctg/ctu-chb/RECORDS.csv")
dataset_records.head()

In [ ]:
# # Create a single outcome file
# def convert_to_int(comment_line: str):
#     value = comment_line.split(' ')[-1]
#     if value != 'NaN':
#         return int(value)
#     else:
#         return float('nan')


def read_and_process_outcome(row):
    record_no: int = int(row["record"])
    record_comments: List[str] = wfdb.rdheader(f"../ctg/ctu-chb/{record_no}").comments
    try:
        ph = float(record_comments[2].split(" ")[-1])
        bcef = float(record_comments[3].split(" ")[-1])
        pco2 = float(record_comments[4].split(" ")[-1])
        be = float(record_comments[5].split(" ")[-1])
        apgar1 = int(record_comments[6].split(" ")[-1])
        apgar5 = int(record_comments[7].split(" ")[-1])
        fetus_age_weeks = int(record_comments[16].split(" ")[-1])
        fetus_weight_grams = float(record_comments[17].split(" ")[-1])
        fetus_sex = int(record_comments[18].split(" ")[-1])
        mother_age_years = int(record_comments[20].split(" ")[-1])
        mother_gravidity = float(record_comments[21].split(" ")[-1])
        mother_parity = int(record_comments[22].split(" ")[-1])
        mother_diabetes = int(record_comments[23].split(" ")[-1])
        mother_hypertension = int(record_comments[24].split(" ")[-1])
        mother_preeclampsia = int(record_comments[25].split(" ")[-1])
        mother_praecox = int(record_comments[26].split(" ")[-1])
        mother_pyrexia = int(record_comments[27].split(" ")[-1])
        mother_meconim = int(record_comments[28].split(" ")[-1])
    except BaseException as e:
        print(record_no)
        print(e)
        raise ValueError()

    return {
        "record_no": record_no,
        "ph": ph,
        "bcef": bcef,
        "pco2": pco2,
        "be": be,
        "apgar1": apgar1,
        "apgar5": apgar5,
        "fetus_age_weeks": fetus_age_weeks,
        "fetus_weight_grams": fetus_weight_grams,
        "fetus_sex": fetus_sex,
        "mother_age_years": mother_age_years,
        "mother_gravidity": mother_gravidity,
        "mother_parity": mother_parity,
        "mother_diabetes": mother_diabetes,
        "mother_hypertension": mother_hypertension,
        "mother_preeclampsia": mother_preeclampsia,
        "mother_praecox": mother_praecox,
        "mother_pyrexia": mother_pyrexia,
        "mother_meconim": mother_meconim,
    }


outcome_df = dataset_records.apply(read_and_process_outcome, axis=1)
outcome_df = pd.DataFrame.from_records(outcome_df.to_list())
outcome_df.to_parquet("../ctg/dataset/outcome.parquet")
outcome_df.head()

In [ ]:
# Convert each record to parquet
def convert_record_to_parquet(row):
    record_no: int = int(row["record_no"])
    record: Record = wfdb.rdrecord(f"../ctg/ctu-chb/{record_no}")
    record_df = record.to_dataframe()
    record_df.to_parquet(f"../ctg/dataset/{record_no}.parquet")


outcome_df.apply(convert_record_to_parquet, axis=1)

In [ ]:
# Delete wfdb db
def delete_unused_items(dir: Path):
    for entry in dir.iterdir():
        if entry.is_dir():
            shutil.rmtree(entry)
        elif entry.is_file():
            entry.unlink()
    dir.rmdir()


wfdb_path = Path("../ctg/ctu-chb/")
delete_unused_items(wfdb_path)